# CMOR 461 / INDE 561 — Homework 8

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/wweissbluth/CMOR-461-LOGISTICS-AND-SUPPLY-CHAIN-MANAGEMENT/blob/main/hw8/hw8.ipynb)


## Problem 1: B&B Bookstore Inventory Optimization

**Scenario:**  
B&B, a local bookstore, must decide how many copies of a new book to order. They can only place one order before the selling season, which lasts one year. Afterward, unsold books are scrapped. 

### Given:

- **Retail price:** $20  
- **Wholesale price (from publisher):** $11  
- **Publisher's raw-material cost:** $3.75  
- **Bookstore shipping & handling cost:** $1.20  
- **Demand:** Normally distributed  
  - Mean: 1200  
  - Standard deviation: 340  
- **Unsold book scrap value:** $0.65  
- **Loss-of-goodwill cost for unmet demand:**  
  - Bookstore: $9.00  
  - Publisher: $4.00  


### Questions:

---

**a)** What is the bookstore’s optimal order quantity?

---

Per the derivation provided in the Canvas files.

We want

$$
\frac{r-w-c_b+g_b}{r-v+g_b}=F(Q^*_r)
$$

Meaning the CDF of the demand must be equal to the `Crtical Ratio`

In [225]:
r = 20 # retail price

w = 11 # wholesale price
c_b = 1.2 # Handling cost

c = 3.75 # production cost

v = .65 # salvage value

g_r = 9 # good will cost (retailer)

# Underage Cost
C_u_r = r - w - c_b + g_r
print(f"Underage Cost: {C_u_r:.2f}")

# Overage Cost
C_o_r = w + c_b - v
print(f"Overage Cost: {C_o_r:.2f}")

critical_ratio_r = (C_u_r) / (C_u_r + C_o_r)
print(f"Critical Ratio: {critical_ratio_r:.2f}")

from scipy.stats import norm

# Inputs
mean_demand = 1200
std_demand = 340

Q_r = norm.ppf(critical_ratio_r, loc=mean_demand, scale=std_demand)

print(f"Optimal book order quantity for retailer {Q_r:.2f} ({int(critical_ratio_r*100)}% of demand)")


Underage Cost: 16.80
Overage Cost: 11.55
Critical Ratio: 0.59
Optimal book order quantity for retailer 1279.63 (59% of demand)


---

**b)** What are the expected profits for the bookstore and publisher using the optimal order quantity from part (a)? What is the total expected profit?

---

`Retailer:`
$$
s(Q) = \mathbb{E}[\min(Q, D)] = \mu - \int_Q^\infty (x - Q) f(x) \, dx = \mu - n(Q)
$$


In [226]:
def q1b():
    from scipy.stats import norm
    from scipy.integrate import quad

    # Parameters
    mu = mean_demand
    sigma = std_demand
    Q = Q_r

    # PDF of normal distribution
    f = lambda x: norm.pdf(x, loc=mu, scale=sigma)

    # Tail integral for n(Q) = ∫_Q^∞ (x - Q) f(x) dx
    integrand = lambda x: (x - Q) * f(x)
    n_Q_r, _ = quad(integrand, Q, float('inf'))

    # Expected sales
    expected_sales = mu - n_Q_r

    # Output
    print(f"Expected lost sales n(Q) = {n_Q_r:.2f}")
    print(f"Expected sales s(Q) = {expected_sales:.2f}")
    return expected_sales, n_Q_r

s_Q_r, n_Q_r = q1b()

Expected lost sales n(Q) = 99.53
Expected sales s(Q) = 1100.47


$$I(Q) = \mathbb{E}[\max(0, Q - D)] = \mathbb{E}[Q - \min(Q, D)] = Q - s(Q)$$

In [227]:
I_Q_r = Q_r - s_Q_r
print(f"Given order quantity {Q_r:.2f}, expected inventory: {I_Q_r:.2f}")

Given order quantity 1279.63, expected inventory: 179.16


$$\pi_r(Q) = r \cdot s(Q) + v \cdot I(Q) - (w + c_B) \cdot Q - g_B \cdot n(Q)$$

In [228]:
pi_r_Q_r = r*s_Q_r + v*I_Q_r - (w+c_b)*Q_r - g_r*n_Q_r
print(f"Retail price (r): {r}")
print(f"Expected sales (s_Q_r): {s_Q_r:.2f}")
print(f"Salvage value (v): {v}")
print(f"Expected inventory (I_Q_r): {I_Q_r:.2f}")
print(f"Wholesale price (w): {w}")
print(f"Handling cost (c_b): {c_b}")
print(f"Order quantity (Q_r): {Q_r:.2f}")
print(f"Goodwill cost (g_r): {g_r}")
print(f"Expected lost sales (n_Q_r): {n_Q_r:.2f}")

print(f"\n\nGiven order quantity {Q_r:.2f}, expected retailer profit: {pi_r_Q_r:.2f}")

Retail price (r): 20
Expected sales (s_Q_r): 1100.47
Salvage value (v): 0.65
Expected inventory (I_Q_r): 179.16
Wholesale price (w): 11
Handling cost (c_b): 1.2
Order quantity (Q_r): 1279.63
Goodwill cost (g_r): 9
Expected lost sales (n_Q_r): 99.53


Given order quantity 1279.63, expected retailer profit: 5618.64


`Supplier:`

$$\pi_s(Q) = (w - c) \cdot Q-g_p\cdot n(Q)

$$


In [229]:
g_s = 4

pi_w_Q_r = (w-c)*Q_r-g_s*n_Q_r
print(f"Given order quantity {Q_r:.2f}, expected supplier profit: {pi_w_Q_r:.2f}")

Given order quantity 1279.63, expected supplier profit: 8879.24


`Whole Supply Chain:`

In [230]:
pi_0_Q_r = pi_r_Q_r + pi_w_Q_r
print(f"Total supply chain profit given order quantity {Q_r:.2f}: {pi_0_Q_r:.2f}")

Total supply chain profit given order quantity 1279.63: 14497.88


---

**c)** What order quantity maximizes total expected profit for the supply chain? What is the optimal total expected profit?

---

$$
C_u = r-(c_b + c)+g_s + g_r 
$$

In [231]:
# Underage Cost - what you lose if you don't have enough stock
c_u_0 = r - (c_b + c) + g_s + g_r 
print(f"\nUnderage Cost for supply chain: {c_u_0:.2f}")



Underage Cost for supply chain: 28.05


$$c_o = -v + (c_b + c)$$

In [232]:
# Overage Cost - what you lose if you have too much stock
c_o_0 = -v + (c_b + c)
print(f"Overage Cost for supply chain: {c_o_0:.2f}")

Overage Cost for supply chain: 4.30


Here we want the critical ratio to be the following:

$$
F_D(Q_r) = \alpha = \frac{C_u}{C_u + C_o}
$$


In [233]:
critical_ratio_0 = (c_u_0) / (c_u_0 + c_o_0)
print(f"\nCritical Ratio for supplier: {critical_ratio_0:.2f}")


Critical Ratio for supplier: 0.87


$$Q_r = \mu + z_\alpha \cdot \sigma$$


In [234]:
Q_0 = norm.ppf(critical_ratio_0, loc=mean_demand, scale=std_demand)
print(f"\nOptimal book order quantity for supply chain {Q_0:.2f} ({int(critical_ratio_0*100)}% of demand)")


Optimal book order quantity for supply chain 1578.31 (86% of demand)


`Retailer:`

$$

s(Q) = \mathbb{E}[\min(Q, D)] = \mu - \int_Q^\infty (x - Q) f(x) \, dx = \mu - n(Q)

$$

In [235]:
def q1b():
    from scipy.stats import norm
    from scipy.integrate import quad

    # Parameters
    mu = mean_demand
    sigma = std_demand
    Q = Q_0

    # PDF of normal distribution
    f = lambda x: norm.pdf(x, loc=mu, scale=sigma)

    # Tail integral for n(Q) = ∫_Q^∞ (x - Q) f(x) dx
    integrand = lambda x: (x - Q) * f(x)
    n_Q_0, _ = quad(integrand, Q, float('inf'))

    # Expected sales
    expected_sales = mu - n_Q_0

    # Output
    print(f"Expected lost sales n(Q) = {n_Q_0:.2f}")
    print(f"Expected sales s(Q) = {expected_sales:.2f}")
    return expected_sales, n_Q_0

s_Q_0, n_Q_0 = q1b()

Expected lost sales n(Q) = 22.75
Expected sales s(Q) = 1177.25


$$I(Q) = \mathbb{E}[\max(0, Q - D)] = \mathbb{E}[Q - \min(Q, D)] = Q - s(Q)$$

In [236]:
I_Q_0 = Q_0 - s_Q_0
print(f"Given order quantity {Q_0:.2f}, expected inventory: {I_Q_0:.2f}")

Given order quantity 1578.31, expected inventory: 401.07


$$\pi_r(Q) = r \cdot s(Q) + v \cdot I(Q) - (w + c_B) \cdot Q - g_B \cdot n(Q)$$

In [237]:
pi_r_Q_0 = r*s_Q_0 + v*I_Q_0 - (w+c_b)*Q_0 - g_r*n_Q_0
print(f"Retail price (r): {r}")
print(f"Expected sales (s_Q_r): {s_Q_r:.2f}")
print(f"Salvage value (v): {v}")
print(f"Expected inventory (I_Q_r): {I_Q_r:.2f}")
print(f"Wholesale price (w): {w}")
print(f"Handling cost (c_b): {c_b}")
print(f"Order quantity (Q_r): {Q_r:.2f}")
print(f"Goodwill cost (g_r): {g_r}")
print(f"Expected lost sales (n_Q_r): {n_Q_r:.2f}")

print(f"\n\nGiven order quantity {Q_r:.2f}, expected retailer profit: {pi_r_Q_r:.2f}")

Retail price (r): 20
Expected sales (s_Q_r): 1100.47
Salvage value (v): 0.65
Expected inventory (I_Q_r): 179.16
Wholesale price (w): 11
Handling cost (c_b): 1.2
Order quantity (Q_r): 1279.63
Goodwill cost (g_r): 9
Expected lost sales (n_Q_r): 99.53


Given order quantity 1279.63, expected retailer profit: 5618.64


`Supplier:`

$$\pi_s(Q) = (w - c) \cdot Q-g_p\cdot n(Q)

$$


In [238]:
pi_w_Q_0 = (w-c)*Q_0-g_s*n_Q_0
print(f"Given order quantity {Q_0:.2f}, expected supplier profit: {pi_w_Q_0:.2f}")

Given order quantity 1578.31, expected supplier profit: 11351.77


`Whole Supply Chain:`

In [239]:
pi_0_Q_0 = pi_r_Q_0 + pi_w_Q_0
print(f"Total supply chain profit given order quantity {Q_0:.2f}: {pi_0_Q_0:.2f}")

Total supply chain profit given order quantity 1578.31: 15697.25


---

**Buyback Contract Scenario:**  
The publisher offers a buyback credit of **$8**.

**d)** What is the bookstore’s optimal order quantity under this buyback contract? What are the expected profits for the bookstore and the publisher?

---

Per the derivation provided in the Canvas files.

We want

$$
\frac{r-w-c_b+g_b}{r-b+g_b}=F(Q^*_r)
$$

where $b$ is the buy back price

Meaning the CDF of the demand must be equal to the `Crtical Ratio`

In [240]:
r = 20 # retail price

b = 8 # buy back price

w = 11 # wholesale price
c_b = 1.2 # Handling cost

c = 3.75 # production cost

v = .65 # salvage value

g_r = 9 # good will cost (retailer)

# Underage Cost
C_u_b = r - w - c_b + g_r
print(f"Underage Cost: {C_u_b:.2f}")

# Overage Cost
C_o_b = w + c_b - b
print(f"Overage Cost: {C_o_b:.2f}")

critical_ratio_b = (C_u_b) / (C_u_b + C_o_b)
print(f"Critical Ratio: {critical_ratio_b:.2f}")

from scipy.stats import norm

# Inputs
mean_demand = 1200
std_demand = 340

Q_b = norm.ppf(critical_ratio_b, loc=mean_demand, scale=std_demand)

print(f"Optimal book order quantity for retailer {Q_b:.2f} ({int(critical_ratio_b*100)}% of demand)")

Underage Cost: 16.80
Overage Cost: 4.20
Critical Ratio: 0.80
Optimal book order quantity for retailer 1486.15 (80% of demand)


`Retailer:`
$$
s(Q) = \mathbb{E}[\min(Q, D)] = \mu - \int_Q^\infty (x - Q) f(x) \, dx = \mu - n(Q)
$$


In [241]:
def q1d():
    from scipy.stats import norm
    from scipy.integrate import quad

    # Parameters
    mu = mean_demand
    sigma = std_demand
    Q = Q_b

    # PDF of normal distribution
    f = lambda x: norm.pdf(x, loc=mu, scale=sigma)

    # Tail integral for n(Q) = ∫_Q^∞ (x - Q) f(x) dx
    integrand = lambda x: (x - Q) * f(x)
    n_Q_b, _ = quad(integrand, Q, float('inf'))

    # Expected sales
    expected_sales = mu - n_Q_b

    # Output
    print(f"Expected lost sales n(Q) = {n_Q_b:.2f}")
    print(f"Expected sales s(Q) = {expected_sales:.2f}")
    return expected_sales, n_Q_b

s_Q_b, n_Q_b = q1d()

Expected lost sales n(Q) = 37.96
Expected sales s(Q) = 1162.04


$$I(Q) = \mathbb{E}[\max(0, Q - D)] = \mathbb{E}[Q - \min(Q, D)] = Q - s(Q)$$

In [242]:
I_Q_b = Q_b - s_Q_b
print(f"Given order quantity {Q_b:.2f}, expected inventory: {I_Q_b:.2f}")

Given order quantity 1486.15, expected inventory: 324.11


$$\pi_r(Q) = r \cdot s(Q) + v \cdot I(Q) - (w + c_B) \cdot Q - g_B \cdot n(Q)$$

In [243]:
pi_r_Q_b = r*s_Q_b + v*I_Q_b - (w + c_b)*Q_b - g_r*n_Q_b
print(f"Retail price (r): {r}")
print(f"Expected sales (s_Q_b): {s_Q_b:.2f}")
print(f"Salvage value (v): {v}")
print(f"Expected inventory (I_Q_b): {I_Q_b:.2f}")
print(f"Wholesale price (w): {w}")
print(f"Handling cost (c_b): {c_b}")
print(f"Order quantity (Q_b): {Q_b:.2f}")
print(f"Goodwill cost (g_r): {g_r}")
print(f"Expected lost sales (n_Q_b): {n_Q_b:.2f}")

print(f"\n\nGiven order quantity {Q_b:.2f}, expected retailer profit: {pi_r_Q_b:.2f}")

Retail price (r): 20
Expected sales (s_Q_b): 1162.04
Salvage value (v): 0.65
Expected inventory (I_Q_b): 324.11
Wholesale price (w): 11
Handling cost (c_b): 1.2
Order quantity (Q_b): 1486.15
Goodwill cost (g_r): 9
Expected lost sales (n_Q_b): 37.96


Given order quantity 1486.15, expected retailer profit: 4978.88


`Supplier:`

$$\pi_s(Q) = (w - c) \cdot Q-g_p\cdot n(Q)

$$


In [244]:
g_s = 4

pi_w_Q_b = (w-c)*Q_b-g_s*n_Q_b
print(f"Given order quantity {Q_b:.2f}, expected supplier profit: {pi_w_Q_b:.2f}")

Given order quantity 1486.15, expected supplier profit: 10622.77


`Whole Supply Chain:`

In [245]:
pi_0_Q_b = pi_r_Q_b + pi_w_Q_b
print(f"Total supply chain profit given order quantity {Q_b:.2f}: {pi_0_Q_b:.2f}")

Total supply chain profit given order quantity 1486.15: 15601.65


---

**e)** What buyback price will coordinate the supply chain? What are the resulting profits?

---

When we say that we want a buy back price to coordinate the supply chain, this means we want the critical ratio for the total supply chain to be equal to that of the retailer.

$$
F(Q_0^*)=F(Q_r^*)
$$

Here we want the critical ratio to be the following:

$$
F(Q_0) = \alpha = \frac{C_u}{C_u + C_o} =\frac{r-(c_b + c)+g_s + g_r }{-v + (c_b + c)+r-(c_b + c)+g_s + g_r }= \frac{r-w-c_b+g_b}{r-b+g_b}=F(Q^*_r)\\

\frac{r-(c_b + c)+g_s + g_r }{-v + (c_b + c)+r-(c_b + c)+g_s + g_r }= \frac{r-w-c_b+g_b}{r-b+g_b}
$$


$$
C_u = r-(c_b + c)+g_s + g_r 
$$

$$C_o = -v + (c_b + c)$$

Note - we don't include the buy-back price here because that's internal to the supply chain, just as we don't include the wholesale price.

In [246]:
from sympy import symbols, Eq, solve
def q1e():
    # Define variables
    r, c_b, c, g_s, g_r, v, w, g_b, b = symbols('r c_b c g_s g_r v w g_b b')

    # Assign values to variables
    r = 20  # retail price
    c_b = 1.2  # handling cost
    c = 3.75  # production cost
    g_s = 4  # goodwill cost (supplier)
    g_r = 9  # goodwill cost (retailer)
    v = 0.65  # salvage value
    w = 11  # wholesale price
    g_b = 9  # goodwill cost (retailer)

    # Define the equation
    lhs = (r - (c_b + c) + g_s + g_r) / (-v + (c_b + c) + r - (c_b + c) + g_s + g_r)
    rhs = (r - w - c_b + g_b) / (r - b + g_b)

    equation = Eq(lhs, rhs)

    # Solve for b
    solution = solve(equation, b)[0]
    print(f"Buyback price (b): {solution}")
    return solution
b = q1e()

Buyback price (b): 9.62459893048129


---

**Revenue Sharing Contract Scenario:**  
The bookstore keeps **60%** of its revenue, and gives **40%** to the publisher.

**f)** What wholesale price will coordinate the supply chain, and what will be the resulting profits?

---

For Retailer:

$$
C_u = 0.6\cdot r - (c_b + w) + g_r \\
C_o = -0.6*v + (c_b + w)\\
F_D(Q_r) = \alpha = \frac{C_u}{C_u + C_o}
$$

For overall supply chain

$$
C_u = r-(c_b + c)+g_s + g_r \\
C_o = -v + (c_b + c)
$$

In [247]:
from sympy import symbols, Eq

def q1f():
    # Define variables
    r, c_b, w, g_r, v = symbols('r c_b w g_r v')
    # Assign values to variables except w
    r = 20  # retail price
    c_b = 1.2  # handling cost
    g_r = 9  # goodwill cost (retailer)
    v = 0.65  # salvage value

    # Underage Cost - what you lose if you don't have enough stock
    c_u_r2 = 0.6 * r - (c_b + w) + g_r
    print(f"\nUnderage Cost for retailer again (symbolic): {c_u_r2}")

    # Overage Cost - what you lose if you have too much stock
    c_o_r2 = -0.6 * v + (c_b + w)
    print(f"Overage Cost for retailer again (symbolic): {c_o_r2}")

    # Critical Ratio
    critical_ratio_r2 = c_u_r2 / (c_u_r2 + c_o_r2)
    print(f"\nCritical Ratio for retailer again (symbolic): {critical_ratio_r2}")

    # Supply Chain Underage Cost
    C_u = r - (c_b + c) + g_s + g_r
    print(f"\nUnderage Cost for supply chain: {C_u:.2f}")

    # Supply Chain Overage Cost
    C_o = -v + (c_b + c)
    print(f"Overage Cost for supply chain: {C_o:.2f}")

    # Supply Chain Critical Ratio
    critical_ratio_02 = C_u / (C_u + C_o)
    print(f"\nCritical Ratio for supply chain: {critical_ratio_02:.2f}")

    # Set the critical ratios equal to solve for w
    equation = Eq(critical_ratio_r2, critical_ratio_02)
    new_whole_sale_price = solve(equation, w)[0]
    print(f"Wholesale price (w) that coordinates the supply chain: {new_whole_sale_price}")
    return new_whole_sale_price, critical_ratio_02

w, critical_ratio_02 = q1f()


Underage Cost for retailer again (symbolic): 19.8 - w
Overage Cost for retailer again (symbolic): w + 0.81

Critical Ratio for retailer again (symbolic): 0.960698689956332 - 0.0485201358563804*w

Underage Cost for supply chain: 28.05
Overage Cost for supply chain: 4.30

Critical Ratio for supply chain: 0.87
Wholesale price (w) that coordinates the supply chain: 1.92950540958269


Notice: the optimal supply chain wholesale price will be:

In [248]:
w

1.92950540958269

But costs are 

In [249]:
c

3.75

Which means this is not a feasible wholesale price. Regardless, this is expected profits:

First ideal order quantity:

In [250]:
critical_ratio_02

0.8670788253477588

In [251]:
from scipy.stats import norm

# Inputs
mean_demand = 1200
std_demand = 340

Q_02 = norm.ppf(critical_ratio_02, loc=mean_demand, scale=std_demand)

print(f"Optimal book order quantity {Q_02:.2f} ({int(critical_ratio_02*100)}% of demand)")

Optimal book order quantity 1578.31 (86% of demand)


profit: 

`Retailer:`
$$
s(Q) = \mathbb{E}[\min(Q, D)] = \mu - \int_Q^\infty (x - Q) f(x) \, dx = \mu - n(Q)
$$


In [252]:
def q1f():
    from scipy.stats import norm
    from scipy.integrate import quad

    # Parameters
    mu = mean_demand
    sigma = std_demand
    Q = Q_02

    # PDF of normal distribution
    f = lambda x: norm.pdf(x, loc=mu, scale=sigma)

    # Tail integral for n(Q) = ∫_Q^∞ (x - Q) f(x) dx
    integrand = lambda x: (x - Q) * f(x)
    n_Q_02, _ = quad(integrand, Q, float('inf'))

    # Expected sales
    expected_sales = mu - n_Q_02

    # Output
    print(f"Expected lost sales n(Q) = {n_Q_02:.2f}")
    print(f"Expected sales s(Q) = {expected_sales:.2f}")
    return expected_sales, n_Q_02

s_Q_02, n_Q_02 = q1f()

Expected lost sales n(Q) = 22.75
Expected sales s(Q) = 1177.25


$$I(Q) = \mathbb{E}[\max(0, Q - D)] = \mathbb{E}[Q - \min(Q, D)] = Q - s(Q)$$

In [253]:
I_Q_02 = Q_02 - s_Q_02
print(f"Given order quantity {Q_02:.2f}, expected inventory: {I_Q_02:.2f}")

Given order quantity 1578.31, expected inventory: 401.07


$$\pi_r(Q) =0.6* r \cdot s(Q) + 0.6*v \cdot I(Q) - (w + c_B) \cdot Q - g_B \cdot n(Q)$$

In [254]:
pi_r_Q_02 = 0.6*(r*s_Q_02 + v*I_Q_02) - (w + c_b)*Q_02 - g_r*n_Q_02
print(f"Retail price (r): {r}")
print(f"Expected sales (s_Q_02): {s_Q_02:.2f}")
print(f"Salvage value (v): {v}")
print(f"Expected inventory (I_Q_02): {I_Q_02:.2f}")
print(f"Wholesale price (w): {w}")
print(f"Handling cost (c_b): {c_b}")
print(f"Order quantity (Q_02): {Q_02:.2f}")
print(f"Goodwill cost (g_r): {g_r}")
print(f"Expected lost sales (n_Q_02): {n_Q_02:.2f}")

print(f"\n\nGiven order quantity {Q_02}, expected retailer profit: {pi_r_Q_02}")

Retail price (r): 20
Expected sales (s_Q_02): 1177.25
Salvage value (v): 0.65
Expected inventory (I_Q_02): 401.07
Wholesale price (w): 1.92950540958269
Handling cost (c_b): 1.2
Order quantity (Q_02): 1578.31
Goodwill cost (g_r): 9
Expected lost sales (n_Q_02): 22.75


Given order quantity 1578.3140005468317, expected retailer profit: 9139.29587156266


`Supplier:`

$$\pi_s(Q) = (w - c) \cdot Q-g_p\cdot n(Q)+(1-0.6)* r \cdot s(Q) + (1-0.6)*v \cdot I(Q)

$$


In [255]:
pi_w_Q_02 = (w-c)*Q_02-g_s*n_Q_02+(1-0.6)*r*s_Q_02+(1-0.6)*v*I_Q_02
print(f"Given order quantity {Q_02:.2f}, expected supplier profit: {pi_w_Q_02:.2f}")

Given order quantity 1578.31, expected supplier profit: 6557.95


`Whole Supply Chain:`

In [256]:
pi_0_Q_b = pi_r_Q_b + pi_w_Q_b
print(f"Total supply chain profit given order quantity {Q_b:.2f}: {pi_0_Q_b:.2f}")

Total supply chain profit given order quantity 1486.15: 15601.65


---

## Problem 2: Two-Stage Ordering in a Supply Chain

**Scenario:**  
A supply chain has a single supplier and single retailer. The retailer can place **two orders**:  
1. Before demand is realized.  
2. After demand is realized (only if the first order is insufficient).  
Any unmet demand before the second order incurs a loss-of-goodwill cost `p`.

### Given:

- Demand is random with:
  - PDF: `f`
  - CDF: `F`
  - Mean: `μ`
- Selling price per item: `r`  
- Wholesale price per item (both orders): `w`  
- Salvage value per unit: `v`  
- First order production cost: `c₁`  
- Second order production cost: `c₂`  
- Assumptions:  
  - `c₁ < c₂`  
  - `v < c₁`  
  - `c₂ < w`  
- The manufacturer **produces to order** (no inventory).

### Sequence of Events:

1. Retailer places **first order** of size `Q`.
2. Order is **delivered immediately**.
3. **Demand is realized**.  
   - Demand ≤ Q: sales fulfilled immediately.  
   - Demand > Q: unmet demand is backordered.
4. Retailer places **second order** for unmet demand.
5. Order is **delivered immediately**, all unmet demand fulfilled.
6. **Unsold inventory** is salvaged.

### Questions:

---

**a)** Write the expected profit functions:
- Retailer’s profit: `π_r(Q)`  
- Supplier’s profit: `π_s(Q)`  
- Total supply chain profit: `Π(Q)`

---


When $I(Q)$ is the ending inventory, $S_1(Q)$ is sales after first round, $S_2(Q)$ is sales to meet missing demand which is equivalent to $n(Q)$ is lost sales after first order: 

$
\pi_r(Q)= (r-w)\cdot S(Q)+(v-w)\cdot I(Q)+(r-w-p)\cdot n(Q)\\
\pi_s(Q)= (w-c_1)\cdot(S(Q)+I(Q))+(w-c_2)\cdot n(Q)\\
\Pi(Q)= (r-c_1)\cdot S(Q)+ (v-c_1)\cdot I(Q)+(r-c_2-p)\cdot n(Q)
$

---

**b)** Prove that the retailer’s optimal order quantity is strictly smaller than the supply chain’s optimal order quantity.  
*Conclusion: the supply chain is not coordinated.*

---

Underage - if you don't have enough stock
Overage - if you have too much stock

Retailer:
$$
C_u = r-w-p\\
C_o = w-v
$$

Given the critical ratio, this becomes $F=\frac{r-w-p}{r-w-p+w-v}=\frac{r-w-p}{r-p-v}$

Supply Chain:
$$
C_u =r-c_2-p\\
C_o = c_1-v
$$

Given the critical ratio, this becomes $F=\frac{r-c_2-p}{r-c_2-p+c_1-v}=\frac{r-c_2-p}{r-p-v+c_1-c_2}$

$$
F=\frac{C_u}{C_u+C_o}
$$

Given $c_1<c_2$, thus $c_1-c_2 <0$ meaning $r-p-v+c_1-c_2 <r-p-v$ ==> this means the supply chain critical ratio should be higher than the retailer.

Given $c_2<w$ ==> the supply chain should be higher

Both numerator and denom. move in the same dir. ==> retailer is strictly lower critical ratio than supply chain ==> retailer orders lower quantity

---

**c)** Consider a **buyback contract**:  
- Retailer pays wholesale price `w`  
- Supplier reimburses the retailer `b` per unsold unit  
- Let:  
  ```math
  w(b) = b + v + (p / (c₁ - v)) * (p + c₂ - c₁)
  ```
  Prove this contract **coordinates the supply chain**.  
  *Verify necessary and sufficient conditions.*

---

Retailer becomes $F=\frac{r-w-p}{r-p-b}$

Supply chain becomes $F=\frac{r-c_2-p}{r-p-v+c_1-c_2}$

In [ ]:
from sympy import symbols, Eq, solve

# Define variables
r, w, p, b, c_2, v, c_1 = symbols('r w p b c_2 v c_1')

# Retailer critical ratio
F_retailer = (r - w - p) / (r - p - b)

# Supply chain critical ratio
F_supply_chain = (r - c_2 - p) / (r - p - v + c_1 - c_2)

# Set the two equations equal
equation = Eq(F_retailer, F_supply_chain)

# Solve for w
solution_w = solve(equation, w)[0].simplify()
print(f"Solution for w: {solution_w}")

################
# Assign values to variables
r_value = 20
p_value = 9
b_value = 9.62459893048129
c_2_value = 4.3
v_value = 0.65
c_1_value = 3.75

# Substitute values into the solution for w
solution_w_value = solution_w.subs({
    r: r_value,
    p: p_value,
    b: b_value,
    c_2: c_2_value,
    v: v_value,
    c_1: c_1_value
}).evalf()

print(f"Notional solved value for w: {solution_w_value}")

Solution for w: (b*c_2 + b*p - b*r + c_1*p - c_1*r - p*v + r*v)/(-c_1 + c_2 + p - r + v)
Solved value for w: 10.0596747790025


In [262]:
from sympy import symbols, simplify

# Define variables
b, v, p, c1, c2 = symbols('b v p c1 c2')

# Define the equation
w_b = b + v + (p / (c1 - v)) * (p + c2 - c1)

# Simplify the equation
w_b_simplified = simplify(w_b)

print(f"Simplified w(b): {w_b_simplified}")

################
# Assign values to variables
r_value = 20
p_value = 9
b_value = 9.62459893048129
c_2_value = 4.3
v_value = 0.65
c_1_value = 3.75

# Substitute values into the solution for w
w_b_simplified = solution_w.subs({
    r: r_value,
    p: p_value,
    b: b_value,
    c_2: c_2_value,
    v: v_value,
    c_1: c_1_value
}).evalf()

print(f"Notional solved value for w(b): {w_b_simplified}")

Simplified w(b): (p*(-c1 + c2 + p) + (b + v)*(c1 - v))/(c1 - v)
Notional solved value for w(b): 10.0596747790025


Clearly they are the same value (as a check), but also the equations are the same as well for $w$.

---

**d)** Briefly explain:
- Why the original supply chain was **not coordinated**.
- Why the **buyback contract** coordinates it.

---

Explanation

### Why the original supply chain was **not coordinated**?
Because the retailer’s and supplier’s incentives were misaligned — the retailer underorders due to a lower critical ratio.
- the original supply chain was not coordinated becuase $w!=c_1!=c_2$

### Why the **buyback contract** coordinates it.
It adjusts the retailer’s incentives (via $ b $) so that the critical ratio matches the supply chain optimum, effectively embedding $ c_1 $, $ c_2 $, and $ p $ into the wholesale terms.
- Buy back fixed this by adjusting the critical ratio such that the equations related, defined by $b(w)$